In [57]:
"""
Mapping.py

Created by John Canty on 02-18-2017

This module is based on Peter Comb's original Python code for aligning two-color data

"""

import numpy as np
import matplotlib.pyplot as plt
import scipy.interpolate as interp

def makeLSQSpline( xl, yl, xr, yr, n):
    """ Creates fitting function for mapping the left channel onto the right channel """
    
    xmin = min(xl)
    xmax = max(xl)
    ymin = min(yl)
    ymax = max(yl)
    
    xknots = np.linspace(xmin, xmax, n)
    yknots = np.linspace(ymin, ymax, n)
    
    xspline = interp.LSQBivariateSpline(xl, yl, xr, xknots, yknots)
    yspline = interp.LSQBivariateSpline(xl, yl, yr, xknots, yknots)
    
    def mapping(xl, yl):
        xr = xspline.ev(xl,yl)
        yr = yspline.ev(xl,yl)
        return xr, yr
    
    mapping.xspline = xspline
    mapping.yspline = yspline
    mapping.maptype = "LSQspline"
    return mapping

if __name__ == '__main__':
    main()

    
    
    



In [49]:
import sys
import argparse

from Mapping import *
import numpy as np
import matplotlib.pyplot as plt
import scipy.interpolate as interp
import xlrd

def main():
    '''Main function docstring'''
    from scipy.io import loadmat
    
    data = loadmat(matfile,struct_as_record = False, squeeze_me = True)
    xl = data[]
    yl = data[]
    xr = data[]
    yr = data[]
    
    reg_dev = []
    
    # Perform mapping 
    for i in range(0, len(xl):
        # Save a point in the grid in both channels
        xl_p = xl[i]
        yl_p = yl[i]
        xr_p = xr[i]
        yr_p = xr[i]
                   
        # Save the rest of the points
        xl_n = xl[:i] + xl[i+1:]
        yl_n = yl[:i] + yl[i+1:]
        xr_n = xr[:i] + xr[i+1:]
        yr_n = yr[:i] + yr[i+1:]

        # Generate the mapping           
        mapping = makeLSQSpline( xl_n, yl_n, xr_n, yr_n, n)
        xr_map, yr_map = mapping(xl_p, yl_p)
        
        # Calculate target registration error
        RMSD = sqrt(abs(xr_map - xr_p)**2 + abs(yr_map - yr_p)**2)                    
        reg_dev = reg_dev.append(RMSD)
                
    mu = mean(reg_dev)
    sigma = std(reg_dev)
    
    # Plot original channels
    plt.plot(xr, yr, 'bo')
    plt.plot(xl, yl, 'ro')
    plt.show()
    
    # Plot registration error histogram
    plt.hist(reg_dev,bins = 15)
    plt.show()
                   
    

    

if __name__ == '__main__':
    main()

SyntaxError: invalid syntax (<ipython-input-49-9abe679b9568>, line 13)